### Team SPHS members:
* Surabhi Sarnot (112584690)
* Priyanka Datar (112681258)
* Himanshu Agrawal (112680639)
* Sri Sneha Geetha Retineni (112671507)

General description:
This code is for the pre-processing of out main dataset and preparing it for further use in our implementation of various concepts.
The output of this file will be the input to county_similarity.py, hypothesis_testing.py and Visualizations.ipynb/Visualizations.py
##### Input files
* college_scorecard.json (converted to CSV seperately "college_scorecard.csv")
* unemployment_csv.csv
* Education_11-18.csv
* Population.csv
* ZIP-COUNTY-FIPS_2018-03.csv

##### Output files
* preprocessed_data.csv


In [408]:
# Importing libraries
from pprint import pprint
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Loading Datasets

In [409]:
# Read Main Data
college_data = pd.read_csv("college_scorecard.csv")
# Read External data
unemployment = pd.read_csv("unemployment_csv.csv")
education = pd.read_csv("Education_11-18.csv")
population = pd.read_csv("Population.csv")
countymap = pd.read_csv("ZIP-COUNTY-FIPS_2018-03.csv")

### Merge External datasets together

In [412]:
population=population.rename(columns={'FIPStxt':'STCOUNTYFP'})
education=education.rename(columns={'FIPS Code':'STCOUNTYFP'})
external_data=pd.merge(education,population,on=['STCOUNTYFP','Year'],how="inner")

In [413]:
external_data_final=pd.merge(unemployment,external_data,on=['STCOUNTYFP','Year'],how="inner")

### Cleaning external dataset

In [414]:
exter_data = pd.DataFrame(external_data_final)

In [415]:
# Dropping columns that are not required
exter_data = exter_data.drop(['Unnamed: 0_x', 'Stabr', 'area_name', 'State_x', 'Area name', 'Unnamed: 0_y','State_y', 'Area_Name','R_birth', 'R_death', 'R_NATURAL_INC',
       'R_INTERNATIONAL_MIG', 'R_DOMESTIC_MIG', 'R_NET_MIG'],axis=1)

In [416]:
# Changing the column dtype and converting comma seperated int values to int
for cols in change_cols:
    exter_data[cols] = exter_data[cols].apply(lambda x: str(x).replace(',',''))
    exter_data[cols] = exter_data[cols].astype('float64')

In [417]:
# Filling the NA for Important columns with mean and max
exter_data['POP_ESTIMATE'].fillna((exter_data['POP_ESTIMATE'].max()), inplace=True)
exter_data["Percent of adults with a bachelor's degree or higher"].fillna((exter_data['Percent of adults with a high school diploma only'].mean()), inplace=True)

In [418]:
# Dropping any remaining NULL values from external dataset
exter_data = exter_data.dropna(how="any")

### Load college data and prepare it by adding ZIPS and Year

In [419]:
# Adding COUNTY ZIP which identifies county uniquely
college_data['ZIP']=college_data['ZIP'].str[0:5]
college_data.dropna(subset=['ZIP'],inplace=True)
college_data['ZIP']=college_data['ZIP'].astype("int64")

In [420]:
# Joining county map to college data to get Unique county identifier in college_data
college_data_fips=pd.merge(college_data,countymap,on="ZIP",how="inner")

In [421]:
college_data_fips=college_data_fips.rename(columns={"YEAR":"Year"})

### Filter college data to year from 2010-2018

In [422]:
# college_filter_year = college_filter[college_filter['Year']>=2010]
college_filter_year = college_data_fips[college_data_fips['Year']>=2010]

In [423]:
# Drop columns greater than 40 % null values
cols = college_filter_year.columns[college_filter_year.isnull().mean()>0.10]
college_filter_col = college_filter_year.drop(cols, axis=1)

In [424]:
# Replace PrivacySuppressed with NaN
college_filter_2 = college_filter_col.replace('PrivacySuppressed', np.nan)

In [425]:
abc = pd.DataFrame(college_filter_2)

In [426]:
# Filtering out rows with more than 5 null values. For better dataset quality
abc = abc[abc.isnull().sum(axis=1) < 5]

In [445]:
abc1 = pd.DataFrame(abc)

In [446]:
# Changing the column dtype and converting comma seperated int values to int
cols = list(abc1.columns)
cols.remove('CITY_y')
cols.remove('STATE')
cols.remove('COUNTYNAME')
cols.remove('CITY_x')
cols.remove('STABBR')
cols.remove('INSTNM')
cols.remove('CLASSFP')
cols.remove('OPEID')
for col in cols:
    abc1[col] = abc1[col].apply(lambda x: str(x).replace(',',''))
    abc1[col] = abc1[col].astype('float64')

In [438]:
# Dropping any missing values entries.
abc1 = abc1.dropna(how="any")

# Now, converting the data by having single entries of (County, Year) and grouping and aggregating by taking the mean for a particular year and county.
college_data_prepared = abc1.groupby(['STCOUNTYFP','Year']).mean().reset_index()

### Merging college data with External Data
* On key (County, Year)
* Doing Inner Joi

In [439]:
prepared_data = pd.merge(college_data_prepared,exter_data,on=['STCOUNTYFP','Year'],how="inner")

In [440]:
# Creating a EQI (Education Quality Index Parameter) which will determine the Quality of Education.
# Taking the parameters based on the Target indicators set for SDG-4 by UN.

In [441]:
prepared_data['perc_total'] = prepared_data['UG12MN']*100/prepared_data['POP_ESTIMATE']
prepared_data['perc_total'].loc[(prepared_data['perc_total'] > 0.493279)] = 0.493279
prepared_data['EQI'] = 1*prepared_data['perc_total'] + 0.4*prepared_data['Percent of adults with a high school diploma only'] + 0.6*prepared_data["Percent of adults completing some college or associate's degree"] + 0.8*prepared_data["Percent of adults with a bachelor's degree or higher"]
prepared_data['EQI'] = prepared_data['EQI']/100

In [444]:
# Writing to csv the file which will be used as input to all the other files in the project.
prepared_data.to_csv("preprocessed_data.csv")